In [ ]:
import rasterio
import scipy.interpolate as spip

def get_get_alt(dem_name):
    dem = rasterio.open(dem_name)
    band = dem.read(1)

    def get_alt(lon, lat):
        r, c = dem.index(lon, lat)
        x, y = dem.xy(r, c)
        dx, dy = dem.res[0], dem.res[1]
        X = [x - dx, x, x + dx]
        Y = [y + dy, y, y - dy]
        Z = band[r-1:r+2, c-1:c+2]
        return spip.interp2d(X, Y, Z)(lon, lat)[0]

    return get_alt

In [ ]:
from shapely.geometry import LineString
from shapely.ops import linemerge

# gdalbuildvrt _hakone.vrt ./dem05s/5239[67][01].tif ./dem05s/5238[67]7.tif
get_alt = get_get_alt('./data/_hakone.vrt')

def get_geom_3d(links):
    geoms = []
    for rid, toward in links:
        row = df[df.rID == rid].iloc[0]
        coords = row.geometry.coords[:]
        coords = coords[::-1] if toward == 'backward' else coords
        if row.railState == 'トンネル':
            a = get_alt(*coords[0])
            geom = LineString([(c[0], c[1], a) for c in coords])
        else:
            geom = LineString([(c[0], c[1], get_alt(*c)) for c in coords])
        geoms.append(geom)
    return linemerge(geoms)

In [ ]:
from shapely.geometry import LineString
from shapely.ops import substring, transform
from pyproj import Transformer

to_meter = Transformer.from_crs('EPSG:4612', 'EPSG:2451', always_xy=True).transform
to_latlon = Transformer.from_crs('EPSG:2451', 'EPSG:4612', always_xy=True).transform

def get_cartgraphic_degrees(geoms, offset, height, timetable):
    def get_distances(geom):
        lengths = [LineString(geom.coords[i:i+2]).length for i in range(len(geom.coords) - 1)]
        dists = [0] + [sum(lengths[:i]) for i in range(2, len(lengths) + 1)]
        adists = [lengths[0] / 2] + [sum(lengths[:i]) for i in range(2, len(lengths))] + [sum(lengths) - lengths[-1] / 2]
        minmax = lambda d: max(min(d, dists[-1]), 0)
        adists = [minmax(d + offset) for d in adists]
        return dists, adists

    def get_section(geom, height, start, end):
        geom = transform(to_meter, geom)
        dists, adists = get_distances(geom)
        fsec = lambda d: (end - start) * (d / dists[-1]) + start
        fpos = lambda d: substring(geom, start_dist=d, end_dist=d).coords[0]
        secs = [fsec(d) for d in dists]
        poss = transform(to_latlon, LineString([fpos(d) for d in adists])).coords
        return [(sec, x, y, z + height) for sec, (x, y, z) in zip(secs, poss)]

    sections = [get_section(geom, height, start, end) for geom, (start, end) in zip(geoms, timetable)]
    return [a for section in sections for pos in section for a in pos]

def get_cartgraphics(geom, height, speed):
    def get_distances(geom):
        lengths = [LineString(geom.coords[i:i+2]).length for i in range(len(geom.coords) - 1)]
        return [0] + [sum(lengths[:i+1]) for i in range(len(lengths))]

    dists = get_distances(transform(to_meter, geom))
    speed_mps = speed * 1000 / 3600
    total_secs = dists[-1] / speed_mps
    secs = [total_secs * (d / dists[-1]) for d in dists]
    sec_coords = [(sec, x, y, get_alt(x, y) + height) for sec, (x, y) in zip(secs, geom.coords)]
    return [a for sec_coord in sec_coords for a in sec_coord]

# cartgraphic_degrees = get_cartgraphics(df.geometry[0], 40.0222 + 2.9805, 50)

In [ ]:
def get_header(name):
    return {
        'id': 'document',
        'name': name,
        'version': '1.0'
    }

def get_body(aid, name, epoch, flat_poss_list, box=None, ellipsoid=None):
    body = {
        'id': aid,
        'name': name,
        'position': {
            'epoch': epoch.strftime('%Y-%m-%dT%H:%M:%S+09'),
            'cartographicDegrees': flat_poss_list
        },
        'orientation': {
            'velocityReference': "%s#position" % aid
        },
        'forwardExtrapolationType': 'None',
        'backwardExtrapolationType': 'None',
    }
    if box:
        body['box'] = box
    if ellipsoid:
        body['ellipsoid'] = ellipsoid
    return body

In [176]:
import json
from datetime import datetime, timedelta
import random as rd
import geopandas as gpd
from pyproj import Transformer
from shapely.ops import substring, transform

to_meter = Transformer.from_crs('EPSG:4612', 'EPSG:2451', always_xy=True).transform
to_latlon = Transformer.from_crs('EPSG:2451', 'EPSG:4612', always_xy=True).transform

# make yumoto down czml
box = {
    'dimensions': { 'cartesian': [4.8, 2, 2] },
    'material': { 'solidColor': { 'color': { 'rgba': [216, 216, 216, 0xff] } } },
    'shadows': 'ENABLED'
}

seeds = [
    [6, 180], [7, 120], [8, 90], [9, 120], [10, 120], [11, 120],
]

def get_secs(hour, interval):
    intervals = [rd.randint(3, interval) for _ in range(300)]
    return [a + hour * 3600 for i, _ in enumerate(intervals) if (a:=sum(intervals[:i])) < 3600]

day = datetime.fromisoformat("2022-11-19")
secs = sum([get_secs(*seed) for seed in seeds], [])
epochs = [day + timedelta(seconds=sec) for sec in secs]

geom = gpd.read_file('./data/_rdcl_1_yumoto.geojson').geometry[0]
for_geom = transform(to_latlon, transform(to_meter, geom).parallel_offset(2))
for_carts = get_cartgraphics(for_geom, 40.0222 + 2.9805, 30)
back_geom = transform(to_latlon, transform(to_meter, geom).parallel_offset(-2))
back_carts = get_cartgraphics(back_geom, 40.0222 + 2.9805, 30)

packets = (
    [get_header('一般車両')] + 
    [get_body(f'{i*2+1}', '上り', epoch, for_carts, box=box) for i, epoch in enumerate(epochs)] +
    [get_body(f'{i*2+2}', '下り', epoch, back_carts, box=box) for i, epoch in enumerate(epochs)]
)
open('./dst/vehicle_yumoto_czml.json', 'w').write(json.dumps(packets, indent=2, ensure_ascii=False))

84599132